# Import Libraries

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import glob
import h5py
from matplotlib.lines import Line2D

In [48]:
def get_cp_edges(thv_array,ny, base1edge, base1_r, thv_threshold1,thv_threshold2,
          cp1_start, cp2_start,base1_speed, cp_disp_thresh_n, cp_disp_thresh_s, cp2_r_adj_init_n, cp2_r_adj_init_s,
         initial_flag_n, initial_flag_s):
    '''This is the outer function that takes an array of virtual potential temperatures, selected virtual
    potential temperature thresholds to define the location of the edge of the cold pools, start times to track the
    cold pools, speed to mone the environmental box, number of points to adjust the cold pool boxes for the 
    environment of cold pool 2, and flags for the north and south edges for edge calculations.
    theta-v array: 3d array of theta-v at the lowest model level above ground in nt,ny,nx 
    cp_center: center esitmate of cold pool 
    base1edge: upper boundary to base state for cp1
    base1_r: size of radius for defining base state for cp1
    thv_threshold_1: theta_v threshold used to define edge of cp1
    thv_threshold_2: theta_v threshold used to define edge of cp2
    cp1_start: start index for tracking cp1
    cp2_start: start index for tracking cp2
    returns:
    cp_edges_n
    cp_edges_s
    thv_prime_1
    thv_prime_2
    lower_box_n
    upper_box_n
    lower_box_s
    upper_box_s
    '''
    ###calculate mean of theta_v through the x dimension
    thv_mean = np.mean(thv_array, axis = 2) 
    thv_bar1 = np.zeros(len(thv_mean[:,1]))
    base1_edge_array = np.zeros(len(thv_mean[:,0]))
    base1_edge_s_array = np.zeros(len(thv_mean[:,0]))
    base1_edge_s = base1edge-base1_r
    
    ### Calculate the environmental average conditions in the specified box for each time step
    for t in range(len(thv_mean[:,0])):
        if base1edge >= len(thv_mean[0,:]):
            base1edge = base1edge-len(thv_mean[0,:])
        if base1_edge_s >= len(thv_mean[0,:]):
            base1_edge_s = base1_edge_s-len(thv_mean[0,:])
        if base1edge > base1_edge_s:
            thv_bar1[t] = np.mean(thv_mean[t,base1_edge_s:base1edge]) ###calculates base state for cp1
        else:
            thv_bar1[t] = np.mean(np.concatenate((thv_mean[t,base1_edge_s:],thv_mean[t,:base1edge])))
        
        base1_edge_array[t] = np.copy(base1edge)
        base1_edge_s_array[t] = np.copy(base1_edge_s)
        
        #update the box coordinates with the base_speed to account for advection
        base1edge = base1edge + base1_speed
        base1_edge_s = base1_edge_s + base1_speed
    
    ### Here, we create offsetting points so that the edge is not located too close to the environmental box for each time step
    offset_n = np.copy(base1_edge_s_array)
    offset_s = np.copy(base1_edge_array)
    offset_n[offset_n<1500] = 0
    offset_n = ny-1-offset_n
    offset_s[offset_s>1500] = 0
    thv_mean_n = thv_mean[:,:] ###splits mean into n and s components relative to cp center
    thv_mean_s = thv_mean[:,:]
    thv_mean_s = np.flip(thv_mean_s, axis =1) ### flips s to be have increasing indicies relative to cp center
    
    ### Calculate the north and south edges
    cp_edges_n, thv_prime_1n, thv_prime_2n, lower_box_n, upper_box_n, cp2_min_ind_n= calc_cpedge(thv_mean_n,thv_bar1, thv_threshold1,
                 thv_threshold2, cp1_start, cp2_start, cp_disp_thresh_n, cp2_r_adj_init_n,initial_flag_n,offset_n)
    cp_edges_s,thv_prime_1s, thv_prime_2s, lower_box_s, upper_box_s, cp2_min_ind_s = calc_cpedge(thv_mean_s, thv_bar1, thv_threshold1,
                 thv_threshold2, cp1_start, cp2_start, cp_disp_thresh_s, cp2_r_adj_init_s, initial_flag_s,offset_s)
    

    cp_edges_s = ny-1-cp_edges_s ### flips the dimensions of southern edge to get it back to same indicies of original array
    lower_box_s = ny-1-lower_box_s
    cp2_min_ind_s = ny-1-cp2_min_ind_s
    
    ### calculated theta_v prime in the domains of CP1 and CP2
    thv_prime_1 = np.concatenate((np.flip(thv_prime_1s, axis =1),thv_prime_1n), axis =1)
    thv_prime_2 = np.concatenate((np.flip(thv_prime_2s, axis =1),thv_prime_2n), axis =1) 
    
    ### Here we are checking to see if the north and south edges of CP1 and CP2 have merged, defined as their edges being a certain
    ### number of points away from each other 
    ns_merge_thresh = 5
    ns_merge_thresh2 = 50
    ss_merge_thresh = 20 ### buffer for initial ss mergers
    if(cp_edges_s[cp2_start-2,0]>cp_edges_n[cp2_start,1]):
        cp_edges_s[:,2] = np.NaN   ###could not merge with south
        thv_prime_1 = thv_mean-thv_bar1[:,None]
        nt = len(thv_prime_1[:,0])
        cp_edges_s[cp2_start-1,0] = np.NaN
        for t in range(cp2_start-1,nt):
            if t== cp2_start-1:
                temp_cp1_edge = np.argmax(thv_prime_1[t,int(cp_edges_n[t+1,1]+ns_merge_thresh):] < thv_threshold1)
                temp_cp1_edge = temp_cp1_edge + int(cp_edges_n[t+1,1]+ns_merge_thresh)
            else:
                temp_cp1_edge = np.argmax(thv_prime_1[t,int(cp_edges_n[t,1]+ns_merge_thresh):] < thv_threshold1)
                if(temp_cp1_edge==0):
                    cp_edges_s[t:,0] = np.NaN
                    if(cp_edges_s[t-1,0]<(cp_edges_n[t,1]+ns_merge_thresh2)):
                        cp_edges_n[t:,2] = np.copy(cp_edges_n[t:,1])
                        break
                    break
                temp_cp1_edge = temp_cp1_edge + int(cp_edges_n[t,1]+ns_merge_thresh)
                
            cp_edges_s[t,0] = temp_cp1_edge
    elif(cp_edges_s[cp2_start-2,0]>cp_edges_s[cp2_start,1]+ss_merge_thresh):
        cp_edges_n[cp2_start:,2]= np.copy(cp_edges_n[cp2_start:,1])
        cp_edges_s[:,2] = np.NaN
        cp_edges_s[cp2_start-1:,0]= np.NaN
        
    return cp_edges_n, cp_edges_s, thv_prime_1, thv_prime_2, lower_box_n, upper_box_n, lower_box_s, upper_box_s,  base1_edge_array, base1_edge_s_array, cp2_min_ind_n, cp2_min_ind_s
    

In [56]:
def calc_cpedge(thv_mean,thv_bar_1, thv_threshold1,thv_threshold2, cp1_start, cp2_start, cp_disp_thresh, cp2_r_adj_init,initial_flag,baseedge):
    '''This function calculates either the north or south edge of CP1 and CP2 for each timestep'''
    
    ###calculates theta-v pertubation for cp1 relative to the base state 1
    thv_prime_1 = thv_mean-thv_bar_1[:,None] 
    ny = np.size(thv_prime_1[0,:])
    nt = len(thv_prime_1[:,0])
    cp1_edges = np.zeros(nt)
    cp1_edges[:] = np.NaN
    
    ###find the location of the CP1 edge for each timestep
    for t in range(cp1_start,nt):
        offset = int(baseedge[t])
        temp_cp1_edge = ny-1-offset-np.argmax(np.flip(thv_prime_1[t,:ny-offset]) < thv_threshold1)   ###generates a point (y) for cp1_edge based on the maximum location where threshold is reached
        if(temp_cp1_edge==(ny-1-offset)):
            cp1_edges[t:] = np.NaN
            break

        cp1_edges[t] = temp_cp1_edge

        #check to see if the cold pool is still propagating forward, if it is not, then stop tracking the edge
        if(cp1_edges[t]<(cp1_edges[t-1]-cp_disp_thresh)): 
            cp1_edges[t:] = np.NaN
            
            break
        
    cp1_edges_int = np.round(cp1_edges).astype(int)   
    cp2_min_ind = np.zeros(nt)
    cp2_min_ind[:] = np.nan
    
    ###calculates an initial base state at the last time where only cp1 is present, #it is the minimum of the inside of cp1 at this time (for the half being calculated)
    base_state2 = np.min(thv_mean[cp2_start-2,:])          
    cp2_min_ind[cp2_start+1] = np.argmin(thv_mean[cp2_start-2,:])
    thv_bar2 = np.zeros(nt)
    thv_prime_2 = np.zeros((nt,ny))
    cp2_edges = np.zeros(nt)
    cp2_edges[:] = np.nan
    cp_merged_edges = np.zeros(nt)
    cp_merged_edges[:] = np.nan
    upper_box = np.zeros(nt)
    upper_box[:] = np.nan
    lower_box = np.zeros(nt)
    lower_box[:] = np.nan
    
    ###calculate the minimum theta_v value and its indicies to calculate the edge of CP2 from
    for t in range(cp2_start,nt):
        if t == cp2_start or t == cp2_start+1:
            thv_bar2[t] = base_state2  ##calculates base state from before cold pool 2 for these 2 timesteps
        else:
            if(np.isnan(cp1_edges[t])):
                thv_bar2[t] = np.min(thv_mean[t,int(lower_box[t]):])#thv_bar2[t] = thv_bar_1[t]
                cp2_min_ind[t] = np.argmin(thv_mean[t,int(lower_box[t]):])+lower_box[t]

            else:
                thv_bar2[t] = np.min(thv_mean[t,int(lower_box[t]):])   ###calculates base state between edge of cp1 and radius base2_r
                cp2_min_ind[t] = np.argmin(thv_mean[t,int(lower_box[t]):]) +lower_box[t]
                
        ###calculate theta_prime
        thv_prime_2[t] = thv_mean[t,:]-thv_bar2[t]
        
        ###calculate the edge of CP2 as the maximum location where theta_v is below the set threshold
        temp_cp2_edge = ny-1-np.argmax(np.flip(thv_prime_2[t,:]) < thv_threshold2)
        
        if temp_cp2_edge == ny-1:
            cp2_edges[t:] = np.nan
            break
        cp2_edges[t] = temp_cp2_edge
         #check to see if the cold pool is still propagating forward, if it is not, then stop tracking the edge
        if(cp2_edges[t]<(cp2_edges[t-1]-cp_disp_thresh)):
            cp2_edges[t:] = np.NaN
            break
                
        # cp2_r_adj_init = 60#30#60   ### The first radius bump in gridpoints for the initial radius guess for the cp2_start+1 time   
        if t == cp2_start+1:
            lower_box[t] = cp2_edges[t]+cp2_r_adj_init
        
        if t > cp2_start and t<nt-1: #nt-2?
            ### Setting an initial guess for the future radius of the base state for cp2
            lower_box[t+1] = lower_box[t]+(cp2_edges[t]-cp2_edges[t-1])#-(cp1_edges[t]-cp1_edges[t-1])
        max_dist_from_cp1= 20 ###this is the minimum number of gridpoints that can be between cp1 edge and cp2 edge        
        
       ###if cp2_edge is greater than or equal to the minimum # of gridpoints between cp edges, they are considered merged
        if(cp2_edges[t]>=(cp1_edges_int[t]-max_dist_from_cp1)):
            # cp2_edges[t+1:] = np.nan
            cp_merged_edges[t+1:] = np.copy(cp1_edges[t+1:])
            cp2_edges[t+1:]= np.copy(cp1_edges[t+1:]) ##this is changed
            cp1_edges[t+1:] = np.nan
            break   #break cp2 loop if the cps have merged
            
        ###checks edges of cp1 an cp2 are close together on the south side for the shear cases so that the edges are properly accounted for
        s_shear_init_merge_thresh = 125
        if initial_flag == True:  ###for south side when  sheared
            if(t==cp2_start):
                if(cp2_edges[t]>=(cp1_edges_int[t]-s_shear_init_merge_thresh)): 
                    cp_merged_edges[t+1:] = np.copy(cp1_edges[t+1:])
                    cp2_edges[t+1:]= np.copy(cp1_edges[t+1:]) ##this is changed
                    cp1_edges[t+1:] = np.nan
                    break
    ###Make a combined array so that the edges of CP1 and CP2 are inside one array, as well as the times where the edges merge
    cp_edges = np.zeros((nt,4))
    cp_edges[:,0] = cp1_edges
    cp_edges[:,1] = cp2_edges
    cp_edges[:,2] = cp_merged_edges
    times = np.arange(len(cp_edges))*5
    cp_edges[:,3] = times
    
    return cp_edges, thv_prime_1, thv_prime_2, lower_box, upper_box, cp2_min_ind
    

In [57]:
def make_cpfile(filePath, cp_edges_n, cp_edges_s, lower_box_n, upper_box_n, lower_box_s, upper_box_s, base1_edge_array, base1_edge_s_array
               ,base1_edge, base1_r, cp2_min_ind_n, cp2_min_ind_s):
    '''makes a file containing timeseries of cp_edges, cp1 environmental box edges, and indicies
    of the location of the environmental minimum points ahead of each CP2 edge'''
    temp_list = [5,10,20]
    time_list = [30,60,90, 120]
    wind_list = ['shear', 'u0']
    ### loop through all parameters to ensure filename matches the right case, then create the file with the data arrays
    for wind in wind_list:
        for CP1_temp in temp_list:
            for CP2_temp in temp_list:
                for time in time_list:
                    if f"CP1_{CP1_temp}k_CP2_{CP2_temp}k_{time}min_{wind}" in filePath:
                        outfilename = f"./cpfile_edges_v7/cpfile_CP1_{CP1_temp}k_CP2_{CP2_temp}k_{time:03}min_{wind}_v2.h5"
                        test_dataset = h5py.File(outfilename, 'w')
                        test_dataset.create_dataset('cp_edges_n', data=cp_edges_n)
                        test_dataset.create_dataset('cp_edges_s', data=cp_edges_s)
                        test_dataset.create_dataset('lower_box_n', data=lower_box_n)
                        test_dataset.create_dataset('lower_box_s', data=lower_box_s)
                        test_dataset.create_dataset('base1_edge_array_n', data=base1_edge_array)
                        test_dataset.create_dataset('base1_edge_array_s', data=base1_edge_s_array)
                        test_dataset.create_dataset('cp2_min_ind_n', data= cp2_min_ind_n)
                        test_dataset.create_dataset('cp2_min_ind_s', data= cp2_min_ind_s)
                        test_dataset.close()

In [58]:
def create_cpedges_dataset(outfilename, base1edge, base1_r):
    '''Outer funtion that takes the filename for each case, the southern edge of the base 
    of the environmental box for CP1, and the width of the bow to make the edge arrays, 
    then puts the edge arrays and assosiated information into a file for each case. 
    It also prints out the filename of each case.
    '''
    ##setting the time to start tracking the second cold pool based on the casename, which is in the filename
    if "30min" in outfilename:
        cp2_start = 8
    elif "60min" in outfilename:
        cp2_start = 14
    elif "90min" in outfilename:
        cp2_start = 20
    elif "120min" in outfilename:
        cp2_start = 26 
    if "shear" in outfilename:
        base1_speed = 33 #10   ###number of gridpoints to move environment box for cp1 per time output
        base1edge = 2050#2800
        ###the number of gridpoints that the edge of the cold pool is allowed to travel backwards between timesteps to see if it is still propagating
        cp_disp_thresh_n = 0   
        cp_disp_thresh_s = 66
        ###This is the number of gridpoints ahead of the cp2 edge at the previous timestep to start looking for the minimum to the edge of 
        ###each side of the domain 
        cp2_r_adj_init_n = 60
        cp2_r_adj_init_s = 60
        initial_flag_n = False
        initial_flag_s = True
    ###If the case is in the NoWind suite, the emvir. box for CP1 does not move and the adjustment and placement thresholds are less
    ### because there is no background advecting winds
    else:
        base1_speed = 0
        cp_disp_thresh_n = cp_disp_thresh_s = 33
        cp2_r_adj_init_n = cp2_r_adj_init_s = 30
        initial_flag_n = initial_flag_s = False
    
    ###read in the surface file for the case
    test_dataset = h5py.File(outfilename, 'r')
    thv = np.array(test_dataset['thv'])
    test_dataset.close()

    ###set the definitions for the minimum virtual potential threshold deficit from the env. to define each cp edge
    thv_threshold1 = -0.5
    thv_threshold2 = -0.3
    cp_center = 1050
    ny = np.size(thv[0,:,0])
    ###always start tracking the CP1 edge at index 2 (10 miniutes after releasing the cold bubble)
    cp1_start = 2
  
    cp_edges_n, cp_edges_s, thv_prime_1, thv_prime_2, lower_box_n, upper_box_n, lower_box_s, upper_box_s, base1_edge_array, base1_edge_s_array, cp2_min_ind_n, cp2_min_ind_s = get_cpedges(thv,ny, base1edge, base1_r, thv_threshold1,
                 thv_threshold2, cp1_start, cp2_start,base1_speed, cp_disp_thresh_n, cp_disp_thresh_s, cp2_r_adj_init_n, cp2_r_adj_init_s, initial_flag_n, initial_flag_s)
    make_cpfile(outfilename, cp_edges_n, cp_edges_s, lower_box_n, upper_box_n, lower_box_s, upper_box_s, base1_edge_array, base1_edge_s_array, base1edge, base1_r, cp2_min_ind_n, cp2_min_ind_s)

    print(outfilename)



In [61]:
###define the paths of the surface data files
filePaths = sorted(glob.glob("./thv*.h5"))

In [62]:
### setting the north edge for the environmental box of CP1 to be y= 280 km for NoWind experiments
b = 2800
###setting the width of the environmental box of CP1 to be 10 km
r = 100
for filePath in filePaths:
    create_cpedges_dataset(filePath,b,r)

/tmp/ipykernel_2354578/1205725665.py:25: RuntimeWarning: invalid value encountered in cast
  cp1_edges_int = np.round(cp1_edges).astype(int)
/tmp/ipykernel_2354578/1205725665.py:74: RuntimeWarning: overflow encountered in scalar subtract
  if(cp2_edges[t]>=(cp1_edges_int[t]-max_dist_from_cp1)):


./thv_CP1_10k_CP2_10k_120min_shear_1.h5
./thv_CP1_10k_CP2_10k_120min_u0_1.h5
./thv_CP1_10k_CP2_10k_30min_shear_1.h5
./thv_CP1_10k_CP2_10k_30min_u0_1.h5
./thv_CP1_10k_CP2_10k_60min_shear_1.h5
./thv_CP1_10k_CP2_10k_60min_u0_1.h5
./thv_CP1_10k_CP2_10k_90min_shear_1.h5
./thv_CP1_10k_CP2_10k_90min_u0_1.h5
./thv_CP1_10k_CP2_20k_120min_shear_1.h5
./thv_CP1_10k_CP2_20k_120min_u0_1.h5
./thv_CP1_10k_CP2_20k_30min_shear_1.h5
./thv_CP1_10k_CP2_20k_30min_u0_1.h5
./thv_CP1_10k_CP2_20k_60min_shear_1.h5
./thv_CP1_10k_CP2_20k_60min_u0_1.h5
./thv_CP1_10k_CP2_20k_90min_shear_1.h5
./thv_CP1_10k_CP2_20k_90min_u0_1.h5
./thv_CP1_10k_CP2_5k_120min_shear_1.h5
./thv_CP1_10k_CP2_5k_120min_u0_1.h5
./thv_CP1_10k_CP2_5k_30min_shear_1.h5
./thv_CP1_10k_CP2_5k_30min_u0_1.h5
./thv_CP1_10k_CP2_5k_60min_shear_1.h5
./thv_CP1_10k_CP2_5k_60min_u0_1.h5
./thv_CP1_10k_CP2_5k_90min_shear_1.h5
./thv_CP1_10k_CP2_5k_90min_u0_1.h5
./thv_CP1_20k_CP2_10k_120min_shear_1.h5
./thv_CP1_20k_CP2_10k_120min_u0_1.h5
./thv_CP1_20k_CP2_10k_30m